Ej. 5. Comparación con búsqueda semántica

Primero instalamos todo el paquete BM25s y lo necesario para la búsqueda semántica ejecutando las siguientes ordenes:

In [ ]:
# To perform lexical search
!pip install bm25s[full]

# To perform semantic search
!pip install chromadb
!pip install sentence_transformers

Descargamos la colección TREC-COVID

In [ ]:
!gdown 1Q1ZnHkstIIQQF8NTPyKQH7sEcpVfThFO
!unzip -o trec-covid-RI.zip

Ahora importamos las librerías Python necesarias para trabajar inicialmente

In [3]:
import bm25s # To create indexes and search documents using BM25
import Stemmer  # For stemming terms
import json  # To load the JSON-formatted corpus

import chromadb
import pickle
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

Seguimos con la preparación del corpus

In [4]:
# Parseamos el corpus
with open("corpus.jsonl", "r", encoding="utf-8") as f:
    corpus_lines = f.readlines() # Leemos todas las lineas del JSONL
    corpus_content = [] # Preparamos una lista de objetos JSON
    for linea in corpus_lines: # Cada línea de un JSONL es un objeto JSON...
        corpus_content.append(json.loads(linea)) # Obtenemos el objeto JSON de la línea i-ésima del JSONL y lo almacenamos en la lista de JSONs

# Preparamos el corpus en los dos formatos requeridos
corpus_verbatim = list() # Una versión literal que mantenga la estructura original de los "documentos".
corpus_plaintext = list() # Una versión de texto plano para tokenización e indexación.
for entry in corpus_content:
    # BM25S expects 'id', 'title', and 'text'
    document = {"id": entry["_id"], "title": entry["title"].lower(), "text": entry["text"].lower()}
    corpus_verbatim.append(document)
    corpus_plaintext.append(entry["text"].lower())

Creamos un stemmer y tokenizamos el corpus

In [ ]:
# Tokenizamos mediante un stemmer para el idioma inglés (PREPROCESADO Y NORMALIZACIÓN)
stemmer = Stemmer.Stemmer("english")
corpus_tokenized = bm25s.tokenize(corpus_plaintext, stopwords="en", stemmer=stemmer, show_progress=True)

Creamos el retriever con la mejor parametrización obtenida del ejercicio 2 (Eliminación de palabras vacías, aplicando estematización y usando la variante de BM25s e IDF lucene), indexamos el corpus ya tokenizado y guardamos el índice generado

In [ ]:
bm25_flavor = "lucene" # Usamos la implementacion lucene de la funcion del IDF
idf_flavor = "lucene" # Usamos la implementacion lucene de la funcion del IDF

retriever = bm25s.BM25(corpus=corpus_verbatim, method=bm25_flavor, idf_method=idf_flavor)
retriever.index(corpus_tokenized, show_progress=True) # Indexamos el corpus tokenizado

#retriever.save("TRECS", corpus=corpus_verbatim) # Guardamos el index generado para poder reutilizarlo llamándolo "trecs"

Ahora preparamos el corpus para ChromaDB (la base de datos vectorial que usaremos)

Obtenemos el texto de los documentos y sus identificadores.

In [7]:
# Prepare documents for ChromaDB
chromadb_documents = corpus_plaintext
chromadb_doc_ids = []

for document in corpus_content:
  doc_id = str(document["_id"])
  chromadb_doc_ids.append(doc_id)

Creamos los embeddings con el siguiente código

In [ ]:
import gzip

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2') # Inicializamos el modelo del transformer de sentencias

crearEmbeddings = False

if crearEmbeddings:
  chromadb_embeddings = model.encode(chromadb_documents, batch_size=100, show_progress_bar=True) #, device='cuda')

  with gzip.open("TREC-embeddings.pickle.gz", "wb") as f:
    pickle.dump(chromadb_embeddings, f)
else:
  !gdown 1iNm1xWo-maxsnkxzlG-hBuqZnFYtCpDe
  !gzip -d TREC-embeddings.pickle.gz

  with open("TREC-embeddings.pickle", "rb") as f:
    chromadb_embeddings = pickle.load(f)

Creamos la colección a almacenar en la base de datos vectorial (ChromaDB)

In [9]:
# Create a persistent ChromaDB client
client = chromadb.PersistentClient(path="./chromadb-storage/")

# We create the collection, please note how we are providing the embedding
# pre-trained model (this is a multilingual model) and we specify the
# distance metric to find the nearest neighbors
collection = client.create_collection(
  name="TREC_collection",
  embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"),
  metadata={"hnsw:space": "cosine"} # https://docs.trychroma.com/guides#changing-the-distance-function
)

Definimos una función para generar lotes

In [10]:
# Función para dividir una lista (de documentos,ids, embeddings) en lotes
# porque ChromaDB puede dar problemas al intentar añadir un dataset grande
# a la colección de una sola vez
def get_batches(lista, chunk_size=100):
  return [lista[i:i + chunk_size] for i in range(0, len(lista), chunk_size)]

Ahora, utilizamos la función anterior para ir creando la colección por lotes:

In [11]:
document_batches = get_batches(chromadb_documents)
ids_batches = get_batches(chromadb_doc_ids)
embedding_batches = get_batches(chromadb_embeddings)

for i in range(len(document_batches)):
  documents = document_batches[i]
  doc_ids = ids_batches[i]
  embeddings = embedding_batches[i]

  # Add the documents, ids and embeddings to the collection
  collection.add(
    documents=documents,
    ids=doc_ids,
    embeddings=embeddings
)

Cargamos las consultas de la colección

In [12]:
queries = []
with open("queries.jsonl", "r", encoding="utf-8") as f:
    queries_lines = f.readlines() # Leemos todas las lineas del JSONL
    for linea in queries_lines: # Cada línea de un JSONL es un objeto JSON...
        queries.append(json.loads(linea)) # Obtenemos el objeto JSON de la línea i-ésima del JSONL y lo almacenamos en la lista de JSONs

Vamos a crear una función para enviar las consultas al motor léxico (bm25s):

In [13]:
# Función para cargar consultas desde un fichero json para enviar a una
# colección dada y obtener listas de resultados.
def submit_queries_to_lexical_and_get_run(queries, stemmer, retriever, max_results=10):

  run = {}

  # Iteramos para enviar cada consulta al recuperador para obtener
  # resultados
  for query in queries:
    # Para evaluar después el rendimiento, es útil conservar el id de
    # la consulta
    query_id = query["_id"]

    query_string = query["metadata"]["narrative"].lower()

    # Tokenizamos la consulta. ¡Atención! Debemos tokenizar la consulta
    # con la misma configuración que usamos al indexar el corpus. En
    # este caso, eliminando las palabras vacías del inglés y aplicando
    # el stemmer de inglés
    #
    query_tokenized = bm25s.tokenize(query_string, stopwords="en", stemmer=stemmer, show_progress=False)

    # Retornamos los k resultados más “top” como una tupla con nombre.
    # Por favor, lee la documentación de BM25S para otras alternativas.
    #
    results = retriever.retrieve(query_tokenized, corpus=retriever.corpus, k=max_results, return_as="tuple", show_progress=False)

    # Los documentos y sus puntuaciones se guardan en dos campos
    # diferentes de la tupla.
    #
    returned_documents = results.documents[0]
    relevance_scores = results.scores[0]

    returned_ids = []
    for i in range(len(returned_documents)):
      ## print(f"\t{i}\t{relevance_scores[i]}\t{
        ## returned_documents[i]['id']}\t
        ## {returned_documents[i]['title'][0:80]}...")
      returned_ids.append(str(returned_documents[i]["id"]))
    run[query_id] = returned_ids

  return run

Conectamos con el motor de búsqueda léxico

In [14]:
import bm25s # Para crear índices y buscar documentos con BM25
import Stemmer # Para estematizar términos

#retriever = bm25s.BM25.load("TRECS", load_corpus=True)
stemmer = Stemmer.Stemmer("english")

Desarrollamos el código de expansión de consultas

In [15]:
import math
from collections import Counter
from heapq import heappush, heappop
from typing import List, Tuple, Any

def calcularTF(texto, tokenizar):
    if tokenizar:
      tokens = bm25s.tokenize(texto, stopwords="en", stemmer=stemmer, show_progress=False)
    else:
      tokens = texto

    tmp = dict()

    for document in tokens[0]:
        freqs = dict(Counter(document))
        for token, freq in freqs.items():
            try:
                tmp[token] += freq
            except:
                tmp[token] = freq

    inverted_vocab = {tokens[1][key]: key for key in tokens[1].keys()}

    total_freqs = dict()

    for key, freq in dict(tmp).items():
        term = inverted_vocab[key]
        total_freqs[term] = freq

    return total_freqs

def obtenerContenidosRelevantes(relevantes):
  contenidoTextual = [] # para cada query guardamos el contenido textual de los documentos resultado

  for i in relevantes: # para cada query tenemos documentos...
    texto = ""
    for j in range(len(relevantes[i])):
      texto += relevantes[i][j]["text"]
    contenidoTextual.append(texto)

  return contenidoTextual

def obtenerFrecuenciasRelevantes(contenidoTextual):
  frecuencias_relevantes = []
  for i in contenidoTextual:
    frecuencias_relevantes.append(calcularTF(i, True))
  return frecuencias_relevantes

def obtenerFrecuenciasRestantes(frecuencias_relevantes, frecuencias_corpus):
  frecuencias_relevantes_restantes = []

  for i in range(len(frecuencias_relevantes)):
    frecuencias_relevantes_restantes.append(dict())

  # recorrer los terminos del corpus
  for termino in frecuencias_corpus:
    frecuencia = frecuencias_corpus[termino]
    # para cada termino buscamos si esta en las frecuencias relevantes de cada query
    for i in range(len(frecuencias_relevantes)):
      conjunto_relevantes = frecuencias_relevantes[i]
      # si el termino del corpus no esta en dicho conjunto i-esimo
      if termino not in conjunto_relevantes.keys():
        # añadimos el termino junto con su frecuencia a la lista de frecuencias restantes en la posicion i-esima
        frecuencias_relevantes_restantes[i][termino] = frecuencia
  return frecuencias_relevantes_restantes

class LogLikelihood:
    @staticmethod
    def entropy(*elements: int) -> float:
        """
        Calculates the unnormalized Shannon entropy:
        -sum x_i log x_i / N = -N sum x_i/N log x_i/N
        """
        total = sum(elements)
        result = sum(LogLikelihood._x_log_x(e) for e in elements)
        return LogLikelihood._x_log_x(total) - result

    @staticmethod
    def _x_log_x(x: int) -> float:
        return 0.0 if x == 0 else x * math.log(x)

    @staticmethod
    def log_likelihood_ratio(k11: int, k12: int, k21: int, k22: int) -> float:
        """
        Calculates the raw log-likelihood ratio for two events.
        """
        assert k11 >= 0 and k12 >= 0 and k21 >= 0 and k22 >= 0, "Counts must be non-negative"

        row_entropy = LogLikelihood.entropy(k11 + k12, k21 + k22)
        column_entropy = LogLikelihood.entropy(k11 + k21, k12 + k22)
        matrix_entropy = LogLikelihood.entropy(k11, k12, k21, k22)

        if row_entropy + column_entropy < matrix_entropy:
            return 0.0  # Round-off error

        return 2.0 * (row_entropy + column_entropy - matrix_entropy)

    @staticmethod
    def root_log_likelihood_ratio(k11: int, k12: int, k21: int, k22: int) -> float:
        """
        Calculates the root log-likelihood ratio for two events.
        """
        llr = LogLikelihood.log_likelihood_ratio(k11, k12, k21, k22)
        sqrt = math.sqrt(llr)
        if (k11 / (k11 + k12)) < (k21 / (k21 + k22)):
            sqrt = -sqrt
        return sqrt

    @staticmethod
    def compare_frequencies(a: Counter, b: Counter, max_return: int, threshold: float) -> List[Tuple[Any, float]]:
        """
        Compares two sets of counts to find over-represented items in the first set.
        """
        total_a = sum(a.values())
        total_b = sum(b.values())
        best = []

        def compare_and_add(t):
            k_a = a[t]
            k_b = b[t]
            score = LogLikelihood.root_log_likelihood_ratio(k_a, total_a - k_a, k_b, total_b - k_b)
            if score >= threshold:
                heappush(best, (score, t))
                if len(best) > max_return:
                    heappop(best)

        for t in a.keys():
            compare_and_add(t)

        if threshold < 0:
            for t in b.keys():
                if t not in a:
                    compare_and_add(t)

        # Sort results by score descending
        return sorted(best, key=lambda x: -x[0])

class ScoredItem:
    def __init__(self, item, score):
        self.item = item
        self.score = score

    def __repr__(self):
        return f"ScoredItem(item={self.item}, score={self.score})"

def obtenerConjuntoValoresLLRS(frecuencias_relevantes, frecuencias_relevantes_restantes):
  numeroMaximoDeTerminos = float('inf') # para que se incluyan todas las puntuaciones
  limiteDePuntuacion = -float('inf') # para que no haya un valor minimo a la hora de puntuar

  conjuntoValoresLLRSigno = []

  for i in range(len(frecuencias_relevantes)):
    conjuntoValoresLLRSigno.append(LogLikelihood.compare_frequencies(Counter(frecuencias_relevantes[i]), Counter(frecuencias_relevantes_restantes[i]), numeroMaximoDeTerminos, limiteDePuntuacion))

  return conjuntoValoresLLRSigno

def obtenerLosMTerminosRelevantesConLosValoresMasAltos(conjuntoValoresLLRSigno, m):
  mTerminosRelevantesConLosValoresMasAltos = []

  for i in range(len(queries)):
    mTerminosRelevantesConLosValoresMasAltos.append(dict())

  frecuenciasTerminosQueries = dict()

  index = 0
  for query in queries:
    frecuenciasTerminosQueries[query["_id"]] = calcularTF(queries[index]["metadata"]["narrative"], True)

    numeroDeTerminosRelevantesEncontrados = 0
    indexValoresLLRS = 0
    while indexValoresLLRS < len(conjuntoValoresLLRSigno[index]) and numeroDeTerminosRelevantesEncontrados < m:
      tupla = conjuntoValoresLLRSigno[index][indexValoresLLRS]
      valor = tupla[0]
      termino = tupla[1]

      if termino not in frecuenciasTerminosQueries[query["_id"]].keys():
        mTerminosRelevantesConLosValoresMasAltos[index][termino] = valor
        numeroDeTerminosRelevantesEncontrados += 1

      indexValoresLLRS += 1

    index += 1
  return mTerminosRelevantesConLosValoresMasAltos

def submit_queries_and_get_run(queries, stemmer, retriever, stopwordsConfig, max_results=100, retornarIdentificadores=False):
    run = {}

    # Iteramos para enviar cada consulta al retriever para obtener
    # resultados
    for query in queries:
        # Para evaluar después el rendimiento, es útil conservar el id de
        # la consulta
        query_id = query["_id"]

        query_string = query["metadata"]["narrative"].lower()

        # Tokenizamos la consulta. ¡Atención! Debemos tokenizar la consulta
        # con la misma configuración que usamos al indexar el corpus.
        #
        query_tokenized = bm25s.tokenize(query_string, stopwords=stopwordsConfig, stemmer=stemmer, show_progress=False)

        # Retornamos los k resultados más “top” como una tupla con nombre.
        # Por favor, lee la documentación de BM25S para otras alternativas.
        #
        results = retriever.retrieve(query_tokenized, corpus=retriever.corpus, k=max_results, return_as="tuple", show_progress=False)

        # Los documentos y sus puntuaciones se guardan en dos campos
        # diferentes de la tupla.
        #
        returned_documents = results.documents[0]
        relevance_scores = results.scores[0]

        returned_content = []
        for i in range(len(returned_documents)):
##          print(f"\t{i}\t{relevance_scores[i]}\t{returned_documents[i]['id']}\t{returned_documents[i]['title'][0:80]}...")
            if retornarIdentificadores:
              returned_content.append(returned_documents[i]["id"])
            else:
              returned_content.append(returned_documents[i])
        run[query_id] = returned_content

    return run

def expandirQueries(queries, n, m):
  # Calculamos las frecuencias de los términos en todos los documentos del corpus
  frecuencias_corpus = calcularTF(corpus_tokenized, False)

  relevantes = submit_queries_and_get_run(queries, stemmer, retriever, "en", n)
  contenidoTextual = obtenerContenidosRelevantes(relevantes)

  frecuencias_relevantes = obtenerFrecuenciasRelevantes(contenidoTextual)
  frecuencias_relevantes_restantes = obtenerFrecuenciasRestantes(frecuencias_relevantes, frecuencias_corpus)

  conjuntoValoresLLRSigno = obtenerConjuntoValoresLLRS(frecuencias_relevantes, frecuencias_relevantes_restantes)

  mTerminosRelevantes = obtenerLosMTerminosRelevantesConLosValoresMasAltos(conjuntoValoresLLRSigno, m)

  queriesExpandidas = queries

  index = 0
  for consulta in queriesExpandidas:
    terminosAñadir = ""
    for termino in mTerminosRelevantes[index]:
      terminosAñadir += " " + termino
    index += 1
    consulta["metadata"]["narrative"] += terminosAñadir
  return queriesExpandidas

Realizamos la expansión de consultas con la mejor configuración obtenida en el ejercicio 4 (n = 1 documento, m = 3 términos)

In [16]:
consultasExpandidas = expandirQueries(queries, 1, 3)

Enviamos las consultas expandidas y originales al motor de búsqueda léxico (bm25s)

In [17]:
expanded_lexical_run = submit_queries_to_lexical_and_get_run(consultasExpandidas, stemmer, retriever)
original_lexical_run = submit_queries_to_lexical_and_get_run(queries, stemmer, retriever)

Conectamos con el motor de búsqueda semántico

In [ ]:
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer


# Crear un cliente ChromaDB persistente
client = chromadb.PersistentClient(path="./chromadb-storage/")

# Obtener la colección disponible en ChromaDB
existing_collections = client.list_collections()

collection_name = "TREC_collection"

# Inicializar el modelo del transformer de sentencias
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Comprobar si la colección existe
if collection_name in [col.name for col in existing_collections]:
    # No tiene mucho sentido que tengamos que indicar qué función de
    # embedding usa la colección, *pero* si no se indica explícitamente,
    # ChromaDB utilizará la función de embedding por defecto y será como
    # comparar manzanas con naranjas…
    collection = client.get_collection(
        collection_name,
        embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    )

    existing_ids = collection.get()["ids"]
    print(f"The collection {collection_name} contains {len(existing_ids)} documents")
else:
    print(f"{collection_name} doesn't exist! You need to create it.")

Vamos a crear una función para enviar las consultas al motor semántico:

In [19]:
# Function to load queries from a json file to submit to a given collection and
# get lists of results
def submit_queries_to_semantic_and_get_run(queries, collection, max_results=10):
    # Initialize the run dictionary
    run = {}

    # Process each query
    for query in queries:
        query_id = query["_id"]

        query_text = query["metadata"]["narrative"].lower()

        # Submit the query to the collection and get the results
        results = collection.query(
            query_texts=[query_text],
            n_results=max_results
        )

        # Store the result IDs in the run dictionary
        run[query_id] = results['ids'][0]

    return run

Enviamos las consultas expandidas y originales al motor de búsqueda semántico

In [20]:
expanded_semantic_run = submit_queries_to_semantic_and_get_run(consultasExpandidas, collection)
original_semantic_run = submit_queries_to_semantic_and_get_run(queries, collection)

Implementamos el código para la carga de los juicios de relevancia

In [21]:
def cargarJuiciosDeRelevancia(recursoJuiciosRelevancia):
    # Leer los datos del archivo de texto
    juicios = {}
    saltarLinea = True
    with open(recursoJuiciosRelevancia, 'r') as f:
        for linea in f:
            if saltarLinea:
                saltarLinea = False
                continue
            if linea.strip():  # Saltar líneas vacías
                query_id, corpus_id, score = linea.split()
                if query_id not in juicios:
                    juicios[query_id] = []
                juicios[query_id].append({"corpus_id": corpus_id, "score": int(score)})

    # El formato de los juicios de relevancia necesita ser ligeramente
    # transformado
    relevance_judgements_reformat = dict()
    for query_id in juicios:
        rel_docs = []
        for j in juicios[query_id]:
            rel_docs.append(j["corpus_id"])
        relevance_judgements_reformat[query_id] = rel_docs

    return relevance_judgements_reformat

juiciosRelevancia = cargarJuiciosDeRelevancia("qrels.tsv")

Y ahora evaluamos el rendimiento obtenido con cada motor de búsqueda.

Primero implementamos el código para medir el rendimiento

In [22]:
def compute_precision_recall_f1(run, relevance_judgements):
    # Initialize lists to hold precision, recall, and f1 scores for each query
    precision_values = []
    recall_values = []
    f1_values = []

    # Initialize global counts for micro-averaging
    global_retrieved = 0
    global_relevant = 0
    global_retrieved_and_relevant = 0

    # Compute precision, recall, and F1 score for each query
    for query_id in run.keys():
        retrieved_results = run[query_id]
        relevant_results = relevance_judgements[query_id]
        relevant_and_retrieved = set(retrieved_results) & set(relevant_results)

        # Update global counts
        global_retrieved += len(retrieved_results)
        global_relevant += len(relevant_results)
        global_retrieved_and_relevant += len(relevant_and_retrieved)

        # Compute precision and recall
        precision = len(relevant_and_retrieved) / len(retrieved_results) if len(retrieved_results) > 0 else 0
        recall = len(relevant_and_retrieved) / len(relevant_results) if len(relevant_results) > 0 else 0

        # Compute F1 score if both precision and recall are non-zero
        if (precision + recall) > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
            f1_values.append(f1)

        # Append precision and recall for the current query
        precision_values.append(precision)
        recall_values.append(recall)

    # Compute macro-averages
    macro_average_precision = sum(precision_values) / len(precision_values) if precision_values else 0
    macro_average_recall = sum(recall_values) / len(recall_values) if recall_values else 0
    macro_average_f1 = sum(f1_values) / len(f1_values) if f1_values else 0

    # Print macro-averaged scores
    print(f"Macro-averaged Precision: {round(macro_average_precision,3)}")
    print(f"Macro-averaged Recall: {round(macro_average_recall,3)}")
    print(f"Macro-averaged F1: {round(macro_average_f1,3)}")
    print("")

    # Compute micro-averages
    micro_average_precision = global_retrieved_and_relevant / global_retrieved if global_retrieved > 0 else 0
    micro_average_recall = global_retrieved_and_relevant / global_relevant if global_relevant > 0 else 0
    micro_average_f1 = (2 * (micro_average_precision * micro_average_recall) /
                        (micro_average_precision + micro_average_recall)) if (micro_average_precision + micro_average_recall) > 0 else 0

    # Print micro-averaged scores
    print(f"Micro-averaged Precision: {round(micro_average_precision,3)}")
    print(f"Micro-averaged Recall: {round(micro_average_recall,3)}")
    print(f"Micro-averaged F1: {round(micro_average_f1,3)}")

Y después llamamos al método anterior con los resultados obtenidos para cada motor de búsqueda

In [ ]:
print("Rendimiento con BM25s (consultas originales): ")
compute_precision_recall_f1(original_lexical_run, juiciosRelevancia)
print()
print("Rendimiento con ChromaDB (consultas originales): ")
compute_precision_recall_f1(original_semantic_run, juiciosRelevancia)

print()
print()

print("Rendimiento con BM25s (consultas expandidas): ")
compute_precision_recall_f1(expanded_lexical_run, juiciosRelevancia)
print()
print("Rendimiento con ChromaDB (consultas expandidas): ")
compute_precision_recall_f1(expanded_semantic_run, juiciosRelevancia)